<img align="right" style="max-width: 200px; height: auto" src="./assets/logo.png">

## Lab 03 - Regelbasierte Analyseverfahren

Lehrgang Internal Auditing, Universität St.Gallen (HSG), 2022

Die Analysen des Seminars **Forensische Datenanalysen** basieren auf Jupyter Notebook (https://jupyter.org). Anhand solcher Notebooks ist es möglich eine Vielzahl von Datenanalysen und statistischen Validierungen durchzuführen. 

<img align="center" style="max-width: 700px" src="./assets/banner.png">

Im Rahmen des vorhergehenden Labs haben wir einige der grundlegenden Vorgehensweisen der Datenvalidierung kennengelernt. In diesem Praktikum werden wir Jupyter Notebook verwenden, um die im Seminar vorgestellten **Regelbasierte Analyseverfahren** praktisch zu vertiefen. Das Hauptziel dieses Notebook ist es, die einzelnen Schritte solcher Analyseverfahren anhand des Beispiels der zu ungewöhnlichen Zeiten erfassten Buchungen durchzuführen.

<img align="center" style="max-width: 800px; height: auto" src="./assets/analytics_process.png">

Im Zweifelsfall oder bei Fragen wenden Sie sich, wie immer gerne an uns via **marco (dot) schreyer (at) unisg (dot) ch**. Wir wünschen Ihnen Viel Freude mit unseren Notebooks und Ihren forensischen Analysen!

## Lernziele des Labs:

Nach der heutigen Übung sollten Sie in der Lage sein:

> 1. Erste eigene Regelbasierte Datenanlysen mit **Jupyter** und **Python** durchzuführen.
> 2. Die Bibliotheken **Pandas** und **Matplotlib** im Kontext Regelbasierter Analyseverfahren anzuwenden.
> 3. Buchungen im Hinblick auf **ungewöhnliche Erfassungszeiten** hypothesenbasiert zu analysieren.
> 4. Erste **konkrete Ideen** für mögliche Regelbasierte Datenanalysen in Ihrem Unternehmen zu entwickeln.

## 1. Einrichtung der Jupyter Notebook-Umgebung

In Analogie zu unserem einführenden Notebook ist es zunächst wieder notwendig, einige Python-Bibliotheken zu importieren, die es uns ermöglichen, Daten zu importieren, zu analysieren und zu visualisieren. In diesem Notebook werden wir hierzu wieder die in **Lab 01** vorgestellten Bibliotheken **(1) Pandas** (https://pandas.pydata.org), **(2) NumPy** (https://numpy.org) und **(3) Matplotlib** (https://matplotlib.org) verwenden.

Lassen Sie uns nun die beiden wichtigsten Datenanalyse-Bibliotheken `Pandas` und `NumPy` entsprechend importieren, indem wir die nachfolgenden beiden `import` Anweisungen ausführen:

In [ ]:
import pandas as pd
import numpy as np

# setzen globaler Pandas Parameter
pd.options.display.max_rows = 500 # allgemeine Darstellung Anzahl Zeilen
pd.options.display.float_format = '{:.2f}'.format # numerische Darstellung von Gleitkommazahlen

Ausserdem importieren wir einige **Utility Bibliotheken**, d.h. Bibliotheken die wichtige zusätzliche Funktionalität zur Verfügung stellen:

In [ ]:
import os # ermöglicht die Erstellung, den Zugriff und die Manipulation von Datenverzeichnissen
import datetime as dt # ermöglicht die Erstellung von Zeitstempeln für Daten

Auch importieren wir eine Reihe von **zusätzlichen Bibliotheken** für den Datenzugriff und den Datenimport in Python:

In [ ]:
import io # ermöglicht das Öffnen und den Zugriff auf Datenströme
import pathlib # ermöglicht das Schreiben von lokaken Dateien
import urllib.request # ermöglicht die Erstellung von Webseiten Anfragen

Schliesslich importieren wir wieder die Bibliothek `Matplotlib` und setzen einige der allgemeinen Parameter für die Datenvisualisierung:

In [ ]:
import matplotlib.pyplot as plt

# setzen globaler Matplotlib Paramater der Datenvisualisierung
plt.style.use('seaborn') # den Visualisierungsstil festlegen
plt.rcParams['figure.figsize'] = [5, 5] # die Visualisierungsgrösse festlegen
plt.rcParams['figure.dpi']= 150 # die Visualisierungsauflösung festlegen

Die nachfolgende Anweisung aktiviert das sog. **Inline-Plotten** von Schaubildern innerhalb des aktuellen Notebooks:

In [ ]:
%matplotlib inline

Darüber hinaus ist es gute Praxis, für jedes Notebook eine entsprechende Ordnerstruktur anzulegen. Diese Ordnerstruktur dient im weiteren Analyseverlauf dazu, sowohl die **Originaldaten** als auch **Analyseergebnisse** und **Schaubilder** zu speichern:

In [ ]:
# erstellen des Verzeichnis der Analysedaten
analysis_data_dir = './02_analysis_data'
if not os.path.exists(analysis_data_dir): os.makedirs(analysis_data_dir)

Abschliessend möchten wir wieder mögliche **Warnungen** einzelner Bibliotheken ignorieren:

In [ ]:
import warnings # ermöglicht die Handhabung von Warnmeldungen

# setzen des Warnfilter-Flags, um Warnungen zu ignorieren
warnings.filterwarnings('ignore')

Solche Warnungen können oftmals aufgrund von aktuellen Bibliothekserweiterungen bzw. -weiterentwicklungen erscheinen. Diese sollen uns jedoch im weiteren Analyseverlauf nicht stören.

## 2. Importieren, Validieren und Aufbereiten des Forensic-Accounting Datensatzes

Wie bereits zuvor in **Lab 02** möchten wir auch diesmal wieder den synthetischen **Forensic Accounting** importieren und für die nachfolgende **Regelbasierten Analysen** aufbereiten. Der synthetische **Forensic Accounting** Datensatz stellt einen Datenextrakt aus SAP Finance, einem der am meisten verbreitetsten ERP-Systeme, dar. Konkret umfasst der Datensatz die beiden grundlegenden Tabellen eines **SAP Finance (FI) Moduls**. Das SAP FI Modul umfasst sämtliche Geschäftsprozesse im Bereich des Finanz- und Rechnungswesens Dazu gehören unter anderem die **Debitoren- und Kreditorenbuchhaltung** sowie die **Haupt- und Nebenbuchhaltung**.

### 2.1. Importieren des Forensic Accounting Datensatzes

Lassen Sie uns nun die in Lab 02 validierten beiden grundlegenden Tabellen der **Belegköpfe (BKPF)** und **Belegsegmente (BSEG)** des Forensic Accounting Datensatzes ([Quelle](https://github.com/mschermann/forensic_accounting)) importieren. Hierzu ist es zunächst notwendig, den Pfad bzw. die Internetadressen der zu importierenden Tabellen zu definieren:

In [ ]:
bkpf_table_url = 'https://raw.githubusercontent.com/mschermann/forensic_accounting/master/BKPF.csv'
bseg_table_url = 'https://raw.githubusercontent.com/mschermann/forensic_accounting/master/BSEG.csv'

Weitere Einzelheiten über diesen Datensatz sind, wie bereits in **Lab 02** beschrieben, in der nachfolgenden Veröffentlichung zu finden: *Schermann, Michael, and Scott R. Boss. "The White-Collar Hacking Contest: A Novel Approach to Teach Forensic Investigations in a Digital World." (2014).* 

Im Sinne guter forensischer Praxis und den damit verbundenen Dokumentationszwecken erzeugen wir auch einen **Zeitstempel des Datendownloads**. Die Erstellung des Zeitstempels erfolgt über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
# definition des Zeitstempels des Datendownloads
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

In einem nächsten Schritt öffnen wir eine Verbindung zu beiden zuvor definierten Internetadressen. Hierzu verwenden wir die Anweisung `request.urlopen` der bereits importierten `URLLib` Bibliothek:

In [ ]:
bkpf_request = urllib.request.urlopen(bkpf_table_url) # erstellen einer Verbindung zur BKPF URL
bseg_request = urllib.request.urlopen(bseg_table_url) # erstellen einer Verbindung zu BSEG URL

Im Anschluss werden die Daten beider Tabellen **unverändert** d.h. im sogenannten **Byte-Format** heruntergeladen:

In [ ]:
bkpf_data_raw = io.BytesIO(bkpf_request.read()) # einlesen der BKPF Daten
bseg_data_raw = io.BytesIO(bseg_request.read()) # einlesen der BSEG Daten

Nachfolgend speichern wir die Daten beider Tabellen erneut auf dem lokalen Dateisystem im CSV-Format. Der zweite Speichervorgang erfolgt innerhalb des bereits zuvor erstellten Ordners für zu **analysierende Originaldaten** mit dem Namen ***02_analysis_data*** ab:

In [ ]:
# definition des Dateinamens der BKPF Analysedaten
bkpf_analysis_file_name = timestamp + '_bkpf_file_analysis.csv'

# definition Dateipfades der BKPF Originaldaten
bkpf_path_analysis = os.path.join(analysis_data_dir, bkpf_analysis_file_name)

# speichern der BKPF Originaldaten
pathlib.Path(bkpf_path_analysis).write_bytes(bkpf_data_raw.getbuffer())

# definition des Dateinamens der BSEG Originaldaten
bseg_analysis_file_name = timestamp + '_bseg_file_analysis.csv'

# definition Dateipfades der BSEG Originaldaten
bseg_path_analysis = os.path.join(analysis_data_dir, bseg_analysis_file_name)

# speicher der BSEG Originaldaten
pathlib.Path(bseg_path_analysis).write_bytes(bseg_data_raw.getbuffer());

In einem nächsten Schritt möchten wir beide Tabellen jeweils wieder als `Pandas` DataFrame in das Notebook importieren. Hierzu ist es notwendig, die beiden im CSV-Format vorliegenden Dateien über die `read_csv` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) der `Pandas` Bibliothek einzulesen:

In [ ]:
bkpf_data = pd.read_csv(bkpf_path_analysis, sep=',', index_col=0, thousands=',') # einlesen der BKPF Tabelle 
bseg_data = pd.read_csv(bseg_path_analysis, sep=',', index_col=0, thousands=',') # einlesen der BSEG Tabelle 

Lassen Sie uns nun auch die **5 ersten Zeilen** der Tabelle **Belegköpfe (BKPF)** anschauen um zu überprüfen, dass die Daten grundsätzlich im richtigen Format d.h. als `Pandas` DataFrame importiert wurden. Hierzu verwenden wir wieder die `head` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)) der `Pandas` Bibliothek: 

In [ ]:
bkpf_data.head(5)

Anschliessend prüfen wir auch die **5 ersten Zeilen** der Tabelle **Belegsegmente (BSEG)** entsprechend:

In [ ]:
bseg_data.head(5)

### 2.2. Validieren der Importierten Daten

In einem nächsten Schritt validieren wir, wie zuvor in **Lab 02**, die Vollständigkeit der importierten Daten. Hierzu ermitteln wir die Anzahl der Datenzeilen und Datenspalten und gleichen diese Information mit der erwarteten Zeilen- und Spaltenanzahl ab. Der Abgleich erfolgt unter Verwendung der `shape` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html)) der `Pandas`Bibliothek. 

Lassen Sie uns hierzu zunächst die Dimensionalität der Tabelle **Belegköpfe (BKPF)** auswerten:

In [ ]:
bkpf_data.shape

Lassen Sie uns nun auch die Dimensionalität der Tabelle **Belegsegmente (BSEG)** auswerten:

In [ ]:
bseg_data.shape

Die eingelesene Tabelle BKPF umfasst insgesamt **3.313 Zeilen** und **113 Spalten**, die Tabelle BSEG umfasst **2.811 Zeilen** und **336 Spalten**. Um die Vollständigkeit der erhaltenen Daten zu gewährleisten gilt es wieder, diese Informationen mit der erhaltenen Datendokumentation abzugleichen.

### 2.3. Aufbereitung der Importierten Daten

Lassen Sie uns nun zunächst, **wie zuvor in Lab 02**, einen eindeutigen Schlüssel (Identifkationsmerkmal) für die beiden Tabellen Belegköpfe (BKPF) und Belegsegmente (BSEG) erstellen. Für beide Tabellen wird der Schlüssel wieder anhand der vier Merkmale **Mandant (MANDT)**, **Buchungskreis (BUKRS)**, **Geschäftsjahr (GJAHR)** und **Belegnummer (BELNR)** gebildet. 

Hierzu wandeln wir zunächst den Datentyp des Feldes Belegnummer (Tabellenfeld: BELNR) in beiden Tabellen wieder in den Typ `int64` (natürliche Zahlen) um. Diese Umwandlung erfolgt um zu gewährleisten, dass das die zu erzeugenden Schlüsselfelder in beiden Tabellen übereinstimmen. Die Umwandlung erfolgt unter Verwendung der `astype` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)) der `Pandas` Bibliothek:

In [ ]:
bkpf_data['BELNR'] = bkpf_data['BELNR'].astype(np.int64)
bseg_data['BELNR'] = bseg_data['BELNR'].astype(np.int64)

Nachfolgend erzeugen wir nun einen solchen eindeutigen Schlüssel in einer neuen Spalte jeweils für die Tabellen BKPF und BSEG. Das Schlüsselfeld soll die Bezeichnung **KEY** tragen und umfasst die Verkettung der nachfolgenden Merkmale in beiden Tabellen:

>- `MANDT`: Client
>- `BUKRS`: Company Code
>- `BELNR`: Accounting Document Number
>- `GJAHR`: Fiscal Year

Für die Erstellung eines solchen Schlüssels werden die einzelnen Tabellenfelder unter Verwendung der `astype` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)) der Pandas Bibliothek in den Datentyp String `str` (Zeichenkette) umgewandelt und anschliessend verknüpft:

In [ ]:
bkpf_data['KEY'] = bkpf_data['MANDT'].astype(str) + '_' + bkpf_data['BUKRS'].astype(str) + '_' + bkpf_data['GJAHR'].astype(str) + '_' + bkpf_data['BELNR'].astype(str)
bseg_data['KEY'] = bseg_data['MANDT'].astype(str) + '_' + bseg_data['BUKRS'].astype(str) + '_' + bseg_data['GJAHR'].astype(str) + '_' + bseg_data['BELNR'].astype(str)

Wie bereits zuvor in Lab 02 möchten wir nun eine Auswahl der für die Analyse relevanten Datenmerkmale der beiden Tabellen Belegköpfe (BKPF) und Belegsegmente (BSEG) festlegen. Die **relevanten Datenmerkmale** werden wieder in Form einer `Liste` in Python pro Tabelle definiert:

In [ ]:
bkpf_fields = ['KEY', 'MANDT', 'BUKRS', 'BELNR', 'GJAHR', 'BLART', 'USNAM', 'TCODE', 'BLDAT', 'CPUDT', 'CPUTM', 'BUDAT', 'BKTXT', 'WAERS']
bseg_fields = ['KEY', 'MANDT', 'BUKRS', 'BELNR', 'GJAHR', 'BUZEI', 'BSCHL', 'DMBTR', 'WRBTR', 'SHKZG', 'HKONT', 'SGTXT', 'KUNNR', 'LIFNR']

In einem nächsten Schritt werden die Tabellen um die nicht für die Analysen notwendigen Datenmerkmale bereinigt: 

In [ ]:
bkpf_data_sel = bkpf_data[bkpf_fields]
bseg_data_sel = bseg_data[bseg_fields]

Lassen Sie uns nun das Filterergebnis anhand der **5 ersten Zeilen** der Tabelle **Belegköpfe (BKPF)** überprüfen:

In [ ]:
bkpf_data_sel.head(5)

Lassen Sie uns das Filterergebnis auch anhand der **5 ersten Zeilen** der Tabelle **Belegsegmente (BSEG)** überprüfen:

In [ ]:
bseg_data_sel.head(5)

### 2.3. Verknüpfung der Importierten Daten

Für die Zielsetzung der nachfolgenden **Regelabasierten Analyseverfahren** sollen, **wie zuvor in Lab 02**, die aufbereiteten Belegkopfdaten (Tabelle: BKPF) und Belegsegmentdaten (Tabelle: BSEG) wieder miteinander verknüpft werden. In einem letzten Schritt der Datenaufbereitung möchten wir deshalb durch die Erstellung einer relationaler Beziehungen die Tabellen BKPF und BSEG miteinander verknüpfen. Die Verknüpfung beider Beleginformationen erfolgt durch die Verwendung der `merge` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)) der `Pandas` Bibliothek:


In [ ]:
bkpf_bseg_sel_merged = bkpf_data_sel.merge(bseg_data_sel, on='KEY', how='inner', suffixes=('_bkpf', '_bseg'))

Lassen Sie uns nun das Ergebnis der Verknüpfung beider Tabellen anhand der **5 ersten Zeilen** der Verknüpfungstabelle `bkpf_bseg_sel_merged` überprüfen:

In [ ]:
bkpf_bseg_sel_merged.head(5)

Über die Anzahl der Einträge innerhalb der Verknüpfungstabelle erhalten wir wieder eine Auskunft darüber, wie viele Verknüpfungen zwischen beiden Tabellen erstellt werden konnten. Lassen Sie uns deshalb die Anzahl der Zeilen innerhalb der Verknüpfungstabelle prüfen:

In [ ]:
bkpf_bseg_sel_merged.shape

Das Ergebnis zeigt, dass für **2.810 Belegsegmente ein zugehöriger Belegkopf gefunden wurde**. Auf dieser Grundlage möchten wir im Nachfolgenden eine erste regelbasierte Analyse implementieren bzw. durchführen.

## 3. Regelbasierte Analyseverfahren

Im Rahmen der forensischen Analyse strukturierter betriebswirtschaftlicher Daten lassen sich grundlegend die nachfolgenden drei Arten von Analyseverfahren unterschieden: **(1) regelbasierte bzw. 'Red-Flag' Analysen**, **(2) Mathematisch-Statistische Analysen** und **(3) Künstliche Intelligenz bzw. Machine Learning Analysen**. Eine schematische Unterscheidung der verschiedenen Analysearten ist innerhalb des nachfolgenden Schaubilds dargestellt:

<img align="center" style="max-width: 800px" src="./assets/analytics_overview.png">

Die verschiedenen Analysearten unterscheiden sich im Wesentlichen anhand ihrer jeweiligen Erwartungshaltung an das Analyseergebnis und das jeweils notwendige Domain oder Analyse Know-how. 

Im Kontext der in diesem Lab betrachteten **Regelbasierten Analyseverfahren** verfügen Analyst:innen eine vergleichsweise hohe Erwartungshaltung bzw. Hypothese an das erwartete Analyseergebnis. Zum Beispiel, das insgesamt maximal 5 SAP-Benutzerkennung im Einkauf buchen oder das Kreditorenrechnungen gewöhnlich keine Beträge höher als EUR 20.000 aufweisen. Darüber hinaus erfordern regelbasierte Analyseverfahren ein vergleichsweise hohes Domain Know-how zur Umsetzung der Analyse. Zum Beispiel, den Prozessablauf im Einkaufsprozess oder Details über das implementierte Berechtigungskonzept. 

Nachfolgend möchten wir nun beispielhaft eine regelbasierte Analyse implementieren. Hierbei handelt es sich um die Analyse sog. Manueller Buchungen ausserhalb der gewöhnlichen Arbeitszeiten. Um eine solche Analyse zu durchzuführen ist es zunächst notwendig das durch die Analyse **(1) adressierte Risko**, **(2) die analysierte Hypothese**, **(3) die notwendigen Daten** und **(4) das Analysevorgehen** zu definieren. Die Definition der verschiedenen Analyseaspekte erfolgte innerhalb der nachfolgenden Tabelle:

<p>
    <table class="table table-bordered table-striped table-hover">
    <col width="150">
        <tr>
            <td><b align="left" style="font-size:16px">ID: </b></td>
            <td><p align="left" style="font-size:16px">Analyse 001</p></td>
        <tr>
            <td><b align="left" style="font-size:16px">Name: </b></td>
            <td><p align="left" style="font-size:16px">Manuelle Buchungen ausserhalb der gewöhnlichen Arbeitszeit.</p></td>
        <tr>
            <td><b align="left" style="font-size:16px">Risiko: </b></td>
            <td><p align="left" style="font-size:16px">Ausserhalb der regulären Arbeitszeiten erfasste (manuelle) Buchungen können Hinweise auf abweichende Geschäftsvorfälle, Einmalsachverhalte und dolose Handlungen darstellen. Täter*innen möchten im Rahmen der Ausübung doloser Handlungen unerkannt bleiben bzw. diese ungestört ausführen. </p></td>
        </tr>
        <tr>
            <td><b align="left" style="font-size:16px">Hypothese: </b></td>
            <td><p align="left" style="font-size:16px">Reguläre (manuelle) Buchungen der Finanzbuchhaltung werden innerhalb der gewöhnlichen Arbeitszeiten zwischen 06:00 Uhr am Morgen und 20:00 am Abend erfasst.</p></td>
        </tr>
        <tr>
            <td><b align="left" style="font-size:16px">Data: </b></td>
            <td><p align="left" style="font-size:16px">Tabellen SAP-FI Belegköpfe (BKPF) und SAP-FI Belegsegmente (BSEG)</p></td>
        </tr>
        <tr>
            <td><b align="left" style="font-size:16px">Vorgehen: </b></td>
            <td><p align="left" style="font-size:16px">Ermittlung von Belegköpfen, die eine Erfassungszeit zwischen 00:00 Uhr am Abend und 04:00 Uhr am Morgen aufweisen. Verknüpfung der identifizierten Belegköpfe mit den jeweiligen Belegsegmenten. Quantifizierung des Betragsvolumens der Buchungen ausserhalb der gewöhnlichen Arbeitszeiten.</p></td>
        </tr>
    </table>
</p>

### 3.1. Erstellung eines Buchungszeitstempels

Um die beabsichtigte Analyse der Buchungen zu ungewöhnlichen Arbeitszeiten durchzuführen, ist es zunächst notwendig, einen entsprechenden Zeitstempel der Buchungen zu erstellen. Nachfolgend reichern wir deshalb die Belegköpfe (Tabelle: BSEG) um ein neues Feld `DATETIME` an. Dieses Feld soll die Felder **Erfassungsdatum (Tabellenfeld: CPUDT)** und **Erfassungszeit (Tabellenfeld: CPUTM)** der Buchungen in einem Feld kombinieren.

Für die Erstellung des Zeitstempels werden die einzelnen Tabellenfelder in den Datentyp String `str` (Zeichenkette) umgewandelt und anschliessend verknüpft. Für die Umwandlung wird wieder die `astype` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)) der `Pandas` Bibliothek verwendet:

In [ ]:
bkpf_bseg_sel_merged['DATETIME'] = bkpf_bseg_sel_merged['CPUDT'].astype(str) + ' ' + bkpf_bseg_sel_merged['CPUTM'].astype(str)

Lassen Sie uns nun die **5 ersten Zeitstempel** der erstellten Spalte `DATETIME` der Tabelle **Belegköpfe (BKPF)** überprüfen:

In [ ]:
bkpf_bseg_sel_merged['DATETIME'].head(5)

In einem nächsten Schritt wandeln wir die erstellten Zeitstempel in den `Pandas` Datentyp `datetime64` um. Das `datetime64` Datenformat umfasst eine Reihe von Funktionalitäten, um nach zeitliche Analysen durchzuführen, zum Beispiel die Berechnung von Zeitdifferenzen bzw. -offsets ([Dokumentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)). Die Umwandlung erfolgt anhand der `to_datetime` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)) der `Pandas`Bibliothek:

In [ ]:
bkpf_bseg_sel_merged['DATETIME'] = pd.to_datetime(bkpf_bseg_sel_merged['DATETIME'])

Lassen Sie uns nun erneut die **5 ersten Zeitstempel** der umgewandelten Spalte `DATETIME` der Tabelle **Belegköpfe (BKPF)** überprüfen:

In [ ]:
bkpf_bseg_sel_merged['DATETIME'].head(5)

In einem letzten Schritt ersetzen wir nun den numerischen `Index` der Tabelle Belegköpfe (BKPF) durch die erzeugten Zeitstempel. Das Ersetzen des Indexes ermöglicht es `Pandas` die Belegköpfe als eine Abfolge von Buchungen im Zeitverlauf zu interpretieren. Das Ersetzen des Index erfolgt anhand der `set_index` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html)) der `Pandas`Bibliothek:

In [ ]:
bkpf_bseg_sel_merged = bkpf_bseg_sel_merged.set_index(bkpf_bseg_sel_merged['DATETIME'])

Lassen Sie uns nun wieder die **5 ersten Belegköpfe** der Tabelle **Belegköpfe (BKPF)** überprüfen:

In [ ]:
bkpf_bseg_sel_merged.head(5)

Anhand der erstellten Zeitstempel im `datetime64` Format innerhalb der Belegköpfe (Tabelle: BKPF) ist es nachfolgend möglich, eine Analyse ungewöhnlicher Buchungszeiten der unterschiedlichen Buchungen durchzuführen. 

### 3.2. Analyse ungewöhnlicher Buchungszeiten

Um die Analyse ungewöhnlicher Buchungszeiten durchzuführen möchten wir zunächst eine Übersicht über die **stündliche Verteilung der Buchungsaktivität im Tagesverlauf** gewinnen. Die Analyse der Verteilung soll einen Aufschluss darüber vermitteln, welche Tageszeitintervalle eine hohe bzw. eine geringe Buchungsaktivität aufweisen. 

Um dieses Ziel zu erreichen, ist es zunächst notwendig, die Stunde einer Buchung aus dem zuvor erstellten Zeitstempel extrahieren. Für die Extraktion dieser Information aus einem gegebenen Zeitstempel verwenden wir die `hour` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.hour.html)) des Datentyps `datetime64` der `Pandas`Bibliothek. Die Extraktion der Stundeninformation für alle Zeitstempel erfolgt anhand der **sog. 'List Comprehensions'** ([Dokumentation](https://docs.python.org/3/tutorial/datastructures.html)) einer gesonderten Technik der Programmiersprache `Python`. Diese Technik ermöglicht es eine bestimmte `Python` Anweisung, iterativ über alle Einträge einer Tabellenspalte auszuführen. 

Nachfolgend verwenden wir diese Technik um die Stundeninformation aller Buchungen in eine gesonderte Tabellenspalte `HOUR` zu extrahieren:

In [ ]:
bkpf_bseg_sel_merged['HOUR'] = [ele.hour for ele in bkpf_bseg_sel_merged['DATETIME']]

Lassen Sie uns nun die **5 ersten Buchungen** der verknüpften Tabelle aus **Belegköpfe (BKPF)** und **Belegsegmenten (BSEG)** anzeigen um die erstellte Spalte `HOUR` zu prüfen:

In [ ]:
bkpf_bseg_sel_merged.head(5)

In einem zweiten Schritt bestimmen wir nun die Anzahl der erfassten Buchung über die unterschiedlichen Tageszeiten bzw. Stunden. Die Ermittlung der Buchungsaktivität pro Stunde erfolgt anhand der bereits in **Lab 02** vorgestellten  `groupby` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)) der `Pandas` Bibliothek. Hierfür werden die einzelnen Buchungen pro Stunde gruppiert und anschliessend pro Gruppe die eindeutigen Schlüssel über die `count` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)) der `Pandas` Bibliothek gezählt:

In [ ]:
bkpf_bseg_sel_merged.groupby(by=['HOUR']).count()['KEY']

Die Analyse zeigt, dass die **überwiegende Mehrheit** der Belegköpfe zwischen **09:00 Uhr und 20:00 Uhr** erfasst wurden. Auch kann zwischen 22:00 Uhr und 00:00 Uhr eine hohe Buchungsaktivtät beobachtet werden. Darüber hinaus wurde eine geringe Anzahl von Buchungen zu **ungewöhnlichen Zeiten** d.h. zwischen **00:00 Uhr und 04:00 Uhr** erfasst. 

Für eine aussagekräftige Dokumentation der gewonnenen Informationen kann es sinnvoll sein, die Verteilung der Buchungsaktivität z.B. anhand eines **Histograms** zu visualisieren. Zur Erstellung eines solchen Diagramms kann auf die Funktionalität der `Matplotlib` Bibliothek zurückgegriffen werden:

In [ ]:
# initialisieren des Diagramms
fig, ax = plt.subplots()

# erstellen des Histograms der Buchungsaktivität
bkpf_bseg_sel_merged.groupby(by=['HOUR']).count()['KEY'].sort_index().plot(ax=ax, kind='bar', color='cornflowerblue', alpha=0.8)

# hinzufügen der Achsenbeschriftungen
plt.ylabel('Posting Count', fontsize=12)
plt.xlabel('Posting Hour (CPUTM)', fontsize=12)

# hinzufügen des Titels
plt.title('Distribution of Posting Hour Activity', fontsize=12);

In einem dritten Schritt ermitteln wir neben der Buchungsaktivität auch das gebuchte Betragsvolumen über die unterschiedlichen Tageszeiten bzw. Stunden. Die Ermittlung der Buchungsaktivität pro Stunde erfolgt erneut anhand der vorgestellten  `groupby` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)) der `Pandas` Bibliothek.  Hierfür werden die einzelnen Buchungen pro Stunde gruppiert und anschliessend der Buchungsbeträge pro Gruppe anhand `sum` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html)) der `Pandas` Bibliothek aufsaldiert:

In [ ]:
bkpf_bseg_sel_merged.groupby(by=['HOUR']).sum()['DMBTR']

Die Analyse bestätigt das bereits erhaltene Bild, dass die **überwiegende Mehrheit** des Betragsvolumens zwischen **09:00 Uhr und 20:00 Uhr** erfasst wurden. Darüber hinaus wurde ein signifikantes Betragsvolumen zwischen 22:00 Uhr und 00:00 Uhr erfasst. Auch wird deutlich dass die geringe Anzahl von Belegköpfen zu **ungewöhnlichen Zeiten**, d.h. zwischen **00:00 Uhr und 04:00 Uhr**, ein signifikantes Betragsvolumen aufweisen. 

Für eine aussagekräftige Dokumentation der gewonnenen Informationen kann es wieder sinnvoll sein, die Verteilung der Buchungsaktivität z.B. anhand eines **Histograms** zu visualisieren. Zur Erstellung eines solchen Diagramms kann erneut auf die Funktionalität der `Matplotlib` Bibliothek zurückgegriffen werden:

In [ ]:
# initialisieren des Diagramms
fig, ax = plt.subplots()

# erstellen des Histograms des betraglichen Buchungsvolumens
bkpf_bseg_sel_merged.groupby(by=['HOUR']).sum()['DMBTR'].sort_index().plot(ax=ax, kind='bar', color='cornflowerblue', alpha=0.8)

# hinzufügen der Achsenbeschriftungen
plt.ylabel('Posting Count', fontsize=12)
plt.xlabel('Posting AMOUNT (DMBTR)', fontsize=12)

# hinzufügen des Titels
plt.title('Distribution of Posting Hour', fontsize=12);

### 3.3. Extraktion auffälliger Buchungen

In einem finalen Schritt sollen nun die zu ungewöhnlichen Zeiten erfassten Buchungen extrahiert werden. Eine solche Extraktion der zwischen **00:00 Uhr und 04:00 Uhr** erfassten Buchungen kann beispielsweise anhand der `between_time` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.between_time.html)) der `Pandas`Bibliothek erfolgen:

In [ ]:
bkpf_bseg_unusual_hours = bkpf_bseg_sel_merged.between_time('00:00:00', '04:00:00')

Lassen Sie uns nun die extrahierten und zugleich zu ungewöhnlichen Zeiten erfassten Buchungen der verknüpften Tabelle aus **Belegköpfe (BKPF)** und **Belegsegmenten (BSEG)** anzeigen. Um die extrahierten Buchungen zu analysieren, möchten wir uns hierzu auf eine Auswahl Buchungsmerkmale (Tabellenfelder) beschränken:

In [ ]:
# erstellen einer Feldauswahl
relevant_fields = ['KEY', 'HOUR', 'MANDT_bkpf', 'BUKRS_bkpf', 'BELNR_bkpf', 'BUZEI', 'GJAHR_bkpf', 'BLART', 'USNAM', 'CPUDT', 'CPUTM', 'BUDAT', 'WAERS', 'BSCHL', 'DMBTR', 'SHKZG', 'HKONT', 'SGTXT', 'LIFNR']

# anzeige ungewöhnlicher Buchungen
bkpf_bseg_unusual_hours[relevant_fields]

Abschliessend möchten wir zudem die gefilterten Buchungen in eine **externe Excel-Datei** auf dem lokalen Dateisystem exportieren. Dies kann beispielsweise für nachgelagerte substanzielle Prüfungshandlungen sinnvoll erscheinen. 

Zu Dokumentationszwecken erzeugen wir zunächst wieder einen **Zeitstempel des Datenexports**. Die Erstellung des Zeitstempels erfolgt über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

Für den tatsächlichen Export der entsprechenden Buchungen kann auf die `to_excel` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html)) der `Pandas` Bibliothek zurückgegriffen werden. Der Speichervorgang erfolgt innerhalb des bereits zuvor erstellten Ordners für **Analyseergebnisse** mit dem Namen ***03_analysis_results***:

In [ ]:
# definition des Dateinamens
file_name = str(timestamp) + '-R1_002_unusal_posting_times.xlsx'

# definition des Dateipfades
file_directory = os.path.join(analysis_data_dir, file_name)

# extraktion der Daten nach Excel
bkpf_bseg_unusual_hours[relevant_fields].to_excel(file_directory, header=True, index=False, sheet_name='Unusual_Time', encoding='utf-8')

## Lab Aufgaben:

Im Ihr wissen zu vertiefen empfehlen wir, die nachfolgenden Übungen zu bearbeiten:

**1. Validierung der verwendeten Benutzerkennungen.**

> Validieren Sie die **verwendeten Benutzerkennungen** (Tabelle: BKPF, Tabellenfeld: USNAM) der zu ungewöhnlichen Zeiten erfassten Buchungen. Erstellen Sie hierzu, auf Grundlage des Extrakts `bkpf_bseg_unusual_hours`, eine Übersicht der enthaltenen Benutzerkennungen. Ermitteln Sie für jede Benutzerkennung die Anzahl erfasster Belegsegmente unter Verwendung der `value_counts` Anweisung der `Pandas` Bibliothek. Extrahieren Sie abschliessend die Übersicht der Benutzerkennungen in eine gesonderte Excel-Datei.

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

**2. Validierung der Kombination verwendeter Lieferanten und Benutzerkennungen.**

> Validieren Sie die Kombination aus **Lieferanten** (Tabelle: BSEG, Tabellenfeld: LIFNR) und **Buchungstexten** (Tabelle: BSEG, Tabellenfeld: SGTXT). Erstellen Sie hierzu, auf Grundlage des Extrakts `bkpf_bseg_unusual_hours`, eine Pivot Übersicht (siehe Lab 02) der Kombinationen aus Lieferanten und Benutzerkennungen. Ermitteln Sie für jede Kombination die **Anzahl erfasster Belegsegmente** unter Verwendung der `pivot_table` Anweisung der `Pandas` Bibliothek. Extrahieren Sie abschliessend Ihre Analyseergebnisse in eine gesonderte Excel-Datei.

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

## Lab Zusammenfassung:

Dieses dritte Lab Notebook umfasst eine schrittweise Einführung in die Vorgehensweise Regelbasierter bzw. 'Red-Flag' Datenanalysen im Kontext forensischer Datenanalysen. Dabei wurde insbesondere der Teilschritt **Datenanalyse** des im Seminars vorgestellten Datenanalyseprozesses behandelt. Die vorgestellten Codebeispiele und Übungen können als Ausgangspunkt für komplexere und Ihre massgeschneiderten Analysen dienen.